The idea here is to try to index some unknown unit cell by using lattice translational symmetry. 

Pick one of the g-vectors at random and then create a new dataset which is shifted by this g-vector. (g-vector is a scattering vector from the sample, one of the points in the reciprocal lattice of the crystal).

Then we look for coincident points between the original and shifted data. 

Points that correspond to overlaps after lattice translation might come from the same lattice as the chosen point, or you might be unlucky and get an overlap.

This is similar to the idea in grainspotter which used rotational symmetry. The interest here is that using a g-vector from the dataset means you are looking for other spots which come from the same lattice as the spot you picked.

The big "problem" is the $N^2$ search of overlapping the original with the shifted data. This is reduced by using a KDTree structure from scipy. You could also mess about with grids (e.g. put the g-vectors onto a fine 3D grid first, now intersect grid points to reduce the problem sizes). In principle you need to repeat for all possible shift vectors in the dataset, but this is probably more like the number of lattices as you can remove peaks when you find lattices.

We can think about using +/- the vector as there is always a Friedel pair in diffraction pattern. Probably better to choose shift vectors where we observed both +/- g from a real experiment.

In [ ]:
import numpy as np, pylab as pl, scipy.spatial
from ImageD11 import indexing

%matplotlib notebook

In [ ]:
def readgv(f):
    i = indexing.indexer()
    i.readgvfile(f)
    return i.gv

In [ ]:
# demo dataset
g3 = readgv( "3.ubi.gve" )
# Note: the omega values are missing in the simulation

In [ ]:
pl.figure()
pl.plot(g3[:,0],g3[:,1],".")

# select a 'random' peak
np.random.seed(10037) # reproducible
j = np.round( g3.shape[0] * np.random.random() ).astype(int)
g3s = g3 - g3[j]
pl.plot(g3s[:,0],g3s[:,1],".",alpha=0.5)
g3s = g3 + g3[j]
pl.plot(g3s[:,0],g3s[:,1],".",alpha=0.5)

In [ ]:
%%time
kd0 = scipy.spatial.cKDTree( g3 )
kd1 = scipy.spatial.cKDTree( g3 + g3[j] )
kd2 = scipy.spatial.cKDTree( g3 - g3[j] )

In [ ]:
%%time
#  Which number to pick for max_distance is a problem
#     ideally maje some error bar for scattering vector
#  ... all the time is in here ?!
sd1 = kd0.sparse_distance_matrix( kd1, max_distance=0.01 ).tocoo()
sd2 = kd0.sparse_distance_matrix( kd2, max_distance=0.01 ).tocoo()

In [ ]:
# distances between pairs of peaks after shifting
peaks = np.concatenate( (sd1.row, sd1.col, sd2.row, sd2.col ))
distances = np.concatenate(( sd1.data, sd2.data) )

In [ ]:
pl.figure()
_=pl.hist(distances,bins=128)

In [ ]:
pl.figure()
pl.plot(g3[:,0],g3[:,1],".", alpha=0.2)
pl.plot(g3[peaks,0],g3[peaks,1],".")